##### DNN 기반 이진분류 모델 구현
- 데이터 셋 : iris.csv
- 피쳐/속성 : 3개 Sepal.Length, Sepal.Width, Petal.Length, Petal.Width
- 타겟/라벨 : 1개 Setosa와 나머지
- 학습-방법 : 지도학습 > 분류
- 알고리즘  : 인공신경망 (ANN) -> MLP,DNN  : 은닉층이 많은 구성
- 프레임워크: Pytorch

[1] 모듈 로딩 및 데이터 준비 <hr>

In [1]:
# 모듈 로딩
# -Model 관련
import torch
import torch.nn as nn
import torch.nn.functional as F # 변수명과 헷갈릴까봐 소문자는 사용 X

from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from torchmetrics.classification import F1Score, BinaryF1Score
from torchmetrics.classification import BinaryConfusionMatrix

from torchinfo import summary

# - Data 및 시각화 관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split 

In [2]:
# 활용 패키지 버전 체크 ==> 사용자 정의 함수로 구현하기
print(f'Pytorch v.{torch.__version__}')
print(f'Pandas v.{pd.__version__}')

Pytorch v.2.4.1
Pandas v.2.0.3


In [3]:
### 데이터 로딩
DATA_FILE = r'C:\Users\KDP-27\Desktop\KDT6\MachineLearning\data\iris.csv'

### CSV >>> DataFrame
irisDF = pd.read_csv(DATA_FILE)

### 확인
irisDF.head(1)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa


In [4]:
### 타겟 변경 => 정수화, 클래스 3개 => 2개
irisDF['variety'].unique()

array(['Setosa', 'Versicolor', 'Virginica'], dtype=object)

In [5]:
# -- 클래스 3개 => 2개

irisDF['variety'] = (irisDF['variety'] == 'Setosa')

In [6]:
irisDF['variety'] = irisDF['variety'].astype('int') 

In [7]:
irisDF['variety'].unique()

array([1, 0])

In [8]:
irisDF.head(1)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,1


[2] 모델 클래스 설계 및 정의 <hr>

- 클래스 목적 : iris 데이터를 학습 및 추론 목적
- 클래스 이름 : IrisRegModel
- 부모 클래스 : nn.Module
- 매 개 변 수 : 층별 입/출력 개수 고정이므로 필요 없음!
- 속 성 필 드 : 
- 기능 / 역할 
    * _ _ _init_ _ _() : 모델 구조 설정
    * forward() : 순방향 학습 <= 오버라이딩(overriding, 상속관계에서만 가능)
- 클래스 구조 
    * 입력층 : 입력  4개(피쳐 수)  출력 10개(퍼셉트론/뉴런 10개 존재)
    * 은닉층 : 입력 10개           출력  5개(퍼셉트론/뉴런 5개 존재)   
    * 출력층 : 입력  5개           출력  1개(퍼셉트론/뉴런 1개 존재 : 2진분류)

- 활성화함수
    * 클래스 형태 ==> nn.MESLoss, nn.ReLU ==> _ _ _init_ _ _() 메서드에서 사용
    * 함수 형태 ==> torch.nn.functional 아래에 ==> forward() 메서드에서 사용

In [10]:
class IrisBCFModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer = nn.Linear(4,10)
        self.hd_layer = nn.Linear(10,5)
        self.out_layer = nn.Linear(5,1)

    # 순방향 학습 진행 메서드
    def forward(self, input_data):
        # - 입력층
        y = self.in_layer(input_data)  # f11w11+f12w12+f13w13+b, ... , f101w101+f102w102+f103w103+b
        y = F.relu(y)                  # relu => y값의 범위 : 0 <= y (sigmoid면 y값은 0에서 1사이가 됨)

        # - 은닉층 : 10개의 숫자 값 (>=0)
        y = self.hd_layer(y)            # f21w21+f22w22+...+f210w210+b, ... , f230w101...+f230w210+b
        y = F.relu(y)                   # relu => y값의 범위 : 0 <= y

        # - 출력층 : 5개의 숫자 값 (>=0) => 회귀이므로 바로 반환(return)
        return F.sigmoid(self.out_layer(y))               # f31W31+...f330W330+b
    

In [11]:
### [테스트] 모델 인스턴스 생성
model = IrisBCFModel()

print(model)

IrisBCFModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=1, bias=True)
)


In [12]:
### [테스트] 모델 사용 메모리 정보 확인
summary(model,input_size=(17,4))

Layer (type:depth-idx)                   Output Shape              Param #
IrisBCFModel                             [17, 1]                   --
├─Linear: 1-1                            [17, 10]                  50
├─Linear: 1-2                            [17, 5]                   55
├─Linear: 1-3                            [17, 1]                   6
Total params: 111
Trainable params: 111
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터_셋 : iris.csv
- 피쳐_개수 : 3개
- 타겟_개수 : 1개
- 클래스이름 : IrisDataset
- 부모클래스 : utils.data.Dataset
- 속성__필드 : featureDF, targetDF, n_rows, n_features
- 필수메서드 : 
    - _ _init_ _(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    - _ _len_ _(self) : 데이터의 개수 반환
    - _ _getitem_ _(self, index) : 특정 인덱스의 피쳐와 타겟 반환

In [13]:
class IrisDataset(Dataset):
    
    def __init__(self, featureDF, targetDF):
        super().__init__()

        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]

    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values) # 시리즈를 array로 만들기위해 values함 
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)

        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [14]:
## [테스트] 데이터셋 인스턴스 생성

# - DataFrame에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]]     # 2D (150, 3)
targetDF = irisDF[irisDF.columns[-1:]]      # 2D (150, 1)

# - 커스텀데이터셋 인스턴스 생성
irisDS = IrisDataset(featureDF, targetDF)

# - 데이터로더 인스턴스 생성
irisDL = DataLoader(irisDS)
for feature, label in irisDL:
    print(feature.shape, label.shape, feature, label)
    break

torch.Size([1, 4]) torch.Size([1, 1]) tensor([[5.1000, 3.5000, 1.4000, 0.2000]]) tensor([[1.]])


[4] 학습 준비
- 학습횟수 : EPOCH <- 처음부터 끝까지 공부하는 단위
- 배치크기 : BATCH_SIZE <- 한번에 학습할 데이터셋 양
- 위치지정 : DEVICE     <- 텐서 저장 및 실행 위치 (GPU/CPU)
-  학습율  : LR 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001 ~ 0.1 사이 (하이퍼파라미터)

In [15]:
### 학습 진행 관련 설정
EPOCH = 1000

BATCH_SIZE = 10 # 전체 데이터 수 확인후 지정 
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001  # 하이퍼파라미터, 값이 적을수록 촘촘하게 내려감

- 인스턴스/객체 : 모델, 데이터셋, 최적화, (,손실함수 , 성능지표) 

In [16]:
#  모델 인스턴스
model=IrisBCFModel()

In [20]:
# DS과 DL인스턴스
# - 학습용, 검증용, 테스트용 데이터분리
X_train, X_test, y_train, y_test = train_test_split(featureDF,targetDF, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)

print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')
print(f'{y_train.value_counts()/y_train.shape[0]}{y_test.value_counts()/y_test.shape[0]}{y_val.value_counts()/y_val.shape[0]}')
print(f'{type(X_train)} {type(X_test)} {type(X_val)}')

# 학습용, 검증용, 테스트용 데이터셋
trainDS=IrisDataset(X_train,y_train)
valDS=IrisDataset(X_val,y_val)
testDS=IrisDataset(X_test,y_test)


# - 학습용 데이터로더 인스턴스
trainDL=DataLoader(trainDS, batch_size=BATCH_SIZE)


(84, 4) (38, 4) (28, 4)
(84, 1) (38, 1) (28, 1)
variety
0          0.702381
1          0.297619
Name: count, dtype: float64variety
0          0.657895
1          0.342105
Name: count, dtype: float64variety
0          0.571429
1          0.428571
Name: count, dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


- 최적화, 손실함수 인스턴스 생성

In [227]:
# 최적화 인스턴스 => w,b 텐서 즉, model.parameters() 전달
optimizer=optim.Adam(model.parameters(), lr=LR)

# 손실함수 인스턴스 => 분류 => 이진분류 BinaryCrossEntropyLoss => BCELoss
#                             예측값은 확률값으로 전달 ==> sigmoid() AF 처리후 전달
reqLoss=nn.BCELoss()

[5] 학습진행

In [ ]:
len(trainDL), trainDL.__len__()

In [ ]:
## 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY = [[], []], [[], []]
CNT = len(trainDL)
print(f'CNT =>  {CNT}')


for epoch in range(EPOCH):

    # 학습 모드로 모델 설정
    model.train()

    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total, score_total=0,0

    for featureTS, targetTS in trainDL:
        # 학습 진행 
        pre_y=model(featureTS)

        # 손실 계산
        loss = reqLoss(pre_y,targetTS)
        loss_total += loss.item()

        # 성능평가 계산
        #score= F1Score(task='binary')(pre_y,targetTS)
        score = BinaryF1Score()(pre_y, targetTS)
        score_total += score.item() # 텐서이기 때문에 .item() 해줘야 정수가 들어감

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포트 당 겅증기능
    # 모델 검증 모드 설정
    model.eval()
    
    with torch.no_grad():

        # 검증 데이터셋
        val_featureTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)

        #추론/평가
        pre_val=model(val_featureTS)

        #손실
        loss_val=reqLoss(pre_val,val_targetTS)
        
        # 성능평가
        score_val=BinaryF1Score()(pre_val, val_targetTS)


    # 에포크 당 손실값과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/CNT)
    SCORE_HISTORY[0].append(score_total/CNT)


    # 손실값과 성능평가값 저장
    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)


    print(f' [{epoch}/{EPOCH}]\n [TRAIN] LOSS : {LOSS_HISTORY[0][-1]}, SCORE : {SCORE_HISTORY[0][-1]}')
    print(f' [VALID] LOSS : {LOSS_HISTORY[1][-1]}, SCORE : {SCORE_HISTORY[1][-1]}')

- 학습결과 체크 => 학습과 검증의 Loss 변화, 성능 변화